In [ ]:
## Manipulación de datos
import pandas as pd
import numpy as np

## Visualización
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

## Modelos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
df = pd.read_csv(r'..\..\data\processed\Merch_ML.csv',index_col='Order_ID')
df.head()

,Order_Date,Buyer_Age,Sales_Price,Shipping_Charges,Sales_per_Unit,Quantity,Total_Sales,Rating,Product_ID,Order_Location,International_Shipping,Buyer_Gender,Product_Category_Clothing,Product_Category_Ornaments,Product_Category_Other
Order_ID,,,,,,,,,,,,,,,
189440,2024-07-21,30,100,0,100,1,100,4,0,15,0,1,1.0,0.0,0.0
187385,2024-07-20,32,100,0,100,1,100,3,0,6,0,1,1.0,0.0,0.0
181844,2024-07-21,26,9,40,49,1,49,2,1,1,1,0,0.0,0.0,1.0
197934,2024-08-19,28,9,0,9,2,18,3,1,18,0,1,0.0,0.0,1.0
122470,2024-01-06,19,10,0,10,3,30,5,2,11,0,0,0.0,0.0,1.0


In [3]:
# variable objetico: Shipping_Charge.
df['Shipping_Charges'].value_counts()

Shipping_Charges
0      5139
40     1167
25      372
70      361
100     184
50      171
Name: count, dtype: int64

Teniendo en cuenta que la variable es numerica discreta, prefiero aplicar un modelo de clasificacion creando una variable binaria.

In [4]:
df['bin_Shipping_Charges'] = (df['Shipping_Charges'] > 0).astype(int)
df['bin_Shipping_Charges'].value_counts(normalize=True)

bin_Shipping_Charges
0    0.695023
1    0.304977
Name: proportion, dtype: float64

In [5]:
X = df.drop(columns=['Shipping_Charges','Order_Date','bin_Shipping_Charges','International_Shipping','Sales_per_Unit'])
y = df['bin_Shipping_Charges']

# Eliminamos variables muy relacionadas para no generar overfitting y hacer un modelo mas realista.

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.20)

In [7]:
esc = StandardScaler()
X_train = esc.fit_transform(X_train)
X_test = esc.transform(X_test)

In [8]:
log = LogisticRegression(class_weight="balanced", random_state=42)
log.fit(X_train, y_train)
y_pred = log.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.93      0.89      0.91      1029
           1       0.77      0.84      0.81       450

    accuracy                           0.88      1479
   macro avg       0.85      0.87      0.86      1479
weighted avg       0.88      0.88      0.88      1479

[[915 114]
 [ 70 380]]


In [9]:
from sklearn.linear_model import SGDClassifier

xdg = SGDClassifier(loss="log_loss", class_weight="balanced", random_state=42)
xdg.fit(X_train, y_train)

y_pred_xdg = xdg.predict(X_test)

print(classification_report(y_test, y_pred_xdg))
print(confusion_matrix(y_test, y_pred_xdg))


              precision    recall  f1-score   support

           0       0.94      0.91      0.93      1029
           1       0.81      0.88      0.84       450

    accuracy                           0.90      1479
   macro avg       0.88      0.89      0.89      1479
weighted avg       0.90      0.90      0.90      1479

[[938  91]
 [ 55 395]]
